## Reading collections from Mondogdb and performing queries with pyspark.

In [1]:
import pyspark
import findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

import findspark
findspark.init()

In [2]:
conf = pyspark.SparkConf().set('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:3.0.1').setMaster('local').setAppName('my app').setAll([("spark.driver.memory","40g"), ("spark.executor.memory","50g")])
sc= SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

##load database and collection from mongo 
df = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/oluwafemi_asg1.zip0").load()
df.show(5)
df.printSchema()
#spark.stop()



+-----+-----------+--------------------+-----+-----+
|  _id|       city|                 loc|  pop|state|
+-----+-----------+--------------------+-----+-----+
|01001|     AGAWAM|[-72.622739, 42.0...|15338|   MA|
|01002|    CUSHMAN|[-72.51565, 42.37...|36963|   MA|
|01005|      BARRE|[-72.108354, 42.4...| 4546|   MA|
|01007|BELCHERTOWN|[-72.410953, 42.2...|10579|   MA|
|01008|  BLANDFORD|[-72.936114, 42.1...| 1240|   MA|
+-----+-----------+--------------------+-----+-----+
only showing top 5 rows



In [4]:
spark

In [7]:
##perform query on the df
df.filter(df["pop"]<='23396').show(5)

+-----+-----------+--------------------+-----+-----+
|  _id|       city|                 loc|  pop|state|
+-----+-----------+--------------------+-----+-----+
|01001|     AGAWAM|[-72.622739, 42.0...|15338|   MA|
|01005|      BARRE|[-72.108354, 42.4...| 4546|   MA|
|01007|BELCHERTOWN|[-72.410953, 42.2...|10579|   MA|
|01008|  BLANDFORD|[-72.936114, 42.1...| 1240|   MA|
|01010|  BRIMFIELD|[-72.188455, 42.1...| 3706|   MA|
+-----+-----------+--------------------+-----+-----+
only showing top 5 rows



In [8]:
df.createOrReplaceTempView("temp")
some_fruit = spark.sql("SELECT *FROM temp WHERE pop= 23396")
some_fruit.show()

+-----+--------+--------------------+-----+-----+
|  _id|    city|                 loc|  pop|state|
+-----+--------+--------------------+-----+-----+
|01013|CHICOPEE|[-72.607962, 42.1...|23396|   MA|
+-----+--------+--------------------+-----+-----+



## Connect to tweet db and collection

In [10]:
df_tweet = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/tweet_file.tweet_stream_").load()
df_tweet.show(5)
df_tweet.printSchema()

+--------------------+--------------------+------+--------------------+
|                 _id|                data|errors|            includes|
+--------------------+--------------------+------+--------------------+
|{62e97965167d0fcc...|{155909486, {null...|  null|{null, null, [{15...|
|{62e97965167d0fcc...|{1235404915515613...|  null|{null, [{12245393...|
|{62e97965167d0fcc...|{15127829, {null}...|  null|{null, [{89501706...|
|{62e97966167d0fcc...|{235714500, {null...|  null|{null, [{15128293...|
|{62e97966167d0fcc...|{2630269581, {nul...|  null|{null, null, [{26...|
+--------------------+--------------------+------+--------------------+
only showing top 5 rows

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- geo: struct (nullable = true)
 |    |    |-- place_id: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- referenced_tweets: array (

In [63]:
##create subtable with selected df
b=df_tweet.select('data.author_id', 'data.text',"data.geo.place_id","data.referenced_tweets.type", 'includes.users.id','includes.users.username','includes.places.full_name')
b.show(5)

+-------------------+--------------------+--------+------------+--------------------+--------------------+---------+
|          author_id|                text|place_id|        type|                  id|            username|full_name|
+-------------------+--------------------+--------+------------+--------------------+--------------------+---------+
|          155909486|https://t.co/HTjo...|    null|        null|         [155909486]|          [cdiviani]|     null|
|1235404915515613184|RT @ImranKhanPTI:...|    null| [retweeted]|[1235404915515613...|[ARazzaq08530293,...|     null|
|           15127829|RT @ShibleyTelham...|    null| [retweeted]|[15127829, 895017...|[garobhai, Shible...|     null|
|          235714500|@MiguelM34149403 ...|    null|[replied_to]|[235714500, 15128...|[Mr_kaiser, Migue...|     null|
|         2630269581|Alguém pra beber ...|    null|        null|        [2630269581]|     [vivisantoss21]|     null|
+-------------------+--------------------+--------+------------+

In [64]:
###show where place_id is not NULL 
b.createOrReplaceTempView("temp")
some_text = spark.sql("SELECT * FROM temp WHERE place_id IS NOT NULL and type is NULL")
some_text.show(10)

+-------------------+--------------------+----------------+----+--------------------+---------------+--------------------+
|          author_id|                text|        place_id|type|                  id|       username|           full_name|
+-------------------+--------------------+----------------+----+--------------------+---------------+--------------------+
|1020108914233348096|1 thing about it ...|795003fb11ee9829|null|[1020108914233348...|    [laiyahyas]|     [New Haven, CT]|
|1369631277771587589|Amanhã volta pra ...|c838cdfdfffe0825|null|[1369631277771587...|[gustavo_freyy]|  [Palmitos, Brasil]|
|          616700246|Que garrón ser K,...|0079bbc151fa56d2|null|         [616700246]|     [dantenob]|[Rosario, Argentina]|
+-------------------+--------------------+----------------+----+--------------------+---------------+--------------------+



In [69]:
###show where place_id is not NULL 
b.createOrReplaceTempView("temp")
some_text = spark.sql("SELECT  text,place_id,full_name FROM temp WHERE full_name IS NOT NULL")
some_text.show(20)

+--------------------+----------------+--------------------+
|                text|        place_id|           full_name|
+--------------------+----------------+--------------------+
|📸📸🤎 https://t....|01bf5c1e56ac186d|[Riyadh, Kingdom ...|
|1 thing about it ...|795003fb11ee9829|     [New Haven, CT]|
|  @morganxhr which?!|011add077f4d2da3|      [Brooklyn, NY]|
|Amanhã volta pra ...|c838cdfdfffe0825|  [Palmitos, Brasil]|
|@lwtshomerry Yeah...|73d1c1c11b675932|    [Chesapeake, VA]|
|@AzginDulEsra Var...|00d58bbe24ee6718|[Gaziantep, Türkiye]|
|https://t.co/UOsl...|a29d2918916e6486|    [Saga-shi, Saga]|
|Que garrón ser K,...|0079bbc151fa56d2|[Rosario, Argentina]|
+--------------------+----------------+--------------------+



In [70]:
b.createOrReplaceTempView("temp")
some_text = spark.sql("SELECT *FROM temp WHERE author_id= 2630269581")
some_text.show()

+----------+--------------------+--------+----+------------+---------------+---------+
| author_id|                text|place_id|type|          id|       username|full_name|
+----------+--------------------+--------+----+------------+---------------+---------+
|2630269581|Alguém pra beber ...|    null|null|[2630269581]|[vivisantoss21]|     null|
+----------+--------------------+--------+----+------------+---------------+---------+

